In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Dropout, SimpleRNN
import numpy as np
from EDA.DataReader import DataReader
from keras.src.legacy.preprocessing.text import Tokenizer
from keras.src.utils import pad_sequences
from sklearn.metrics import accuracy_score
import torch 
from torcheval.metrics.functional import multiclass_f1_score

In [2]:
# dataReader = DataReader("UIT_VFSC")
dataReader = DataReader("") # tập foody_raw
df_train = dataReader.df_train
df_test = dataReader.df_test
df_total =dataReader.df_total
n_labels = int(df_total["label"].max().item() + 1)

In [ ]:
# Tiền xử lý văn bản
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train["corpus"])
max_words = len(tokenizer.word_index) + 1 
sequences = tokenizer.texts_to_sequences(df_train["corpus"])
X_train = pad_sequences(sequences)
y_train = np.array(df_train["label"])
X_Test = pad_sequences(tokenizer.texts_to_sequences(df_test["corpus"]))
y_test = np.array(df_test["label"])
# Xây dựng mô hình RNN
model = tf.keras.models.Sequential([
    Embedding(input_dim=max_words, output_dim=128),
    SimpleRNN(128),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(n_labels, activation='softmax')  
])

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=10)
y_pred = model.predict(X_Test)
y_pred_classes = np.argmax(y_pred, axis=1)
acc = accuracy_score(y_test, y_pred_classes)

print(f"Accuracy: {acc:.4f}")

y_pred_classes = torch.tensor(y_pred_classes).type(torch.long)
label = torch.tensor(df_test["label"].tolist()).type(torch.long)
mf1 = multiclass_f1_score(y_pred_classes, label, num_classes=n_labels, average='macro')
wf1 = multiclass_f1_score(y_pred_classes, label, num_classes=n_labels, average='weighted')
print(f"F1-Score: {max(mf1, wf1):.4f}")


Epoch 1/10
494/957 ━━━━━━━━━━━━━━━━━━━━ 6:16 814ms/step - accuracy: 0.5430 - loss: 0.6948